<a href="https://colab.research.google.com/github/CheilaBaiao/GEE_SR/blob/main/Pantanal_p%C3%B3s_processamento_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# =========================
# PÓS-PROCESSAMENTO OFFLINE
# NDVI mensal (12 bandas/ano) → 1º break por pixel
# Saídas: CSVs + plots (sem salvar GeoTIFF)
# =========================

# !pip -q install rasterio numpy pandas ruptures matplotlib



In [ ]:
# ---------- Imports ----------
import os, re, math, gc
from pathlib import Path
import numpy as np
import pandas as pd
import rasterio as rio
from rasterio.windows import Window
import ruptures as rpt
import matplotlib.pyplot as plt

from google.colab import drive
drive.mount('/content/drive')  # use o padrão
print("Drive montado!")

In [ ]:
from pickle import FALSE
# ---------- Parâmetros principais ----------
INPUT_DIR  = Path("/content/drive/MyDrive/Pantanal_TippingPoints")  # pasta com os TIFFs anuais
OUT_DIR    = Path("/content/drive/MyDrive/Pantanal_TippingPoints_OFFLINE")  # saídas
OUT_DIR.mkdir(parents=True, exist_ok=True)
PLOTS_DIR  = OUT_DIR / "plots"
PLOTS_DIR.mkdir(exist_ok=True)

# Padrão para capturar o ANO no nome do arquivo (ajuste se necessário)
# Ex.: Pantanal_1985.tif, NDVI_1985_12bands.tif, etc.
YEAR_REGEX = re.compile(r"(19|20)\d{2}")

# Dados e controle
DATA_SCALE     = 10000.0       # NDVI foi salvo como Int16 * 10000
NODATA         = -32768
TILE_PX        = 64            # janela de processamento (pixels) — ajuste 32/64/96
VALID_FRAC_MIN = 0.02          # pula tiles com <2% de válidos
RUN_SAMPLE_ONLY = False         # True: roda amostra; False: roda tudo
SAMPLE_N_TILES  = 20           # nº de tiles na amostra
TOP_N_PLOTS     = 10           # nº de tiles (maior frac_breaks) para plotar

# Ruptures
MODEL_RUPTURES = "l2"          # "l2" é leve e bom após dessazonalizar
MIN_POINTS     = 36            # mínimo de pontos válidos para tentar detectar 1 quebra

In [ ]:
# ---------- Utilitários ----------
def list_year_files(input_dir: Path):
    files = sorted([f for f in input_dir.glob("*.tif") if f.is_file()])
    pairs = []
    for f in files:
        m = YEAR_REGEX.search(f.name)
        if m:
            pairs.append((int(m.group()), f))
    pairs = sorted(pairs, key=lambda x: x[0])
    if not pairs:
        raise RuntimeError("Não encontrei GeoTIFFs anuais no diretório informado.")
    years, paths = zip(*pairs)
    return list(years), list(paths)

def check_alignment(ref_ds, other_ds):
    same = (
        ref_ds.crs == other_ds.crs and
        ref_ds.transform == other_ds.transform and
        ref_ds.width == other_ds.width and
        ref_ds.height == other_ds.height and
        ref_ds.count == ref_ds.count  # só para simetria; deixamos checar bandas à parte
    )
    return same

def month_index_to_ym(idx, years):
    """Mapeia índice 0..(len(years)*12-1) para (YYYY, MM)."""
    y = years[idx // 12]
    m = 1 + (idx % 12)
    return int(y), int(m)

def _fill_na_linear_1d(x: np.ndarray):
    idx = np.arange(x.size)
    good = np.isfinite(x)
    if good.sum() < 2:
        return None
    return np.interp(idx, idx[good], x[good])

def deseasonalize_series_matrix(mat_2d, n_years):
    """mat_2d shape: (Npix, T) com T = 12*n_years; retorna mesma shape."""
    T = mat_2d.shape[1]
    assert T == n_years * 12, "Esperava T múltiplo de 12."
    out = mat_2d.copy()
    # climatologia por mês (média por coluna m::12)
    for m in range(12):
        cols = np.arange(m, T, 12)
        clim = np.nanmean(out[:, cols], axis=1, keepdims=True)
        out[:, cols] = out[:, cols] - clim
    return out

def first_break_1d(ts: np.ndarray, model=MODEL_RUPTURES):
    x = _fill_na_linear_1d(ts.astype("float32"))
    if x is None or np.isfinite(x).sum() < MIN_POINTS:
        return -1
    try:
        bkps = rpt.Binseg(model=model).fit(x).predict(n_bkps=1)
        bp = bkps[0] - 1 if bkps and bkps[0] < len(x) else -1
        return int(bp)
    except Exception:
        return -1

def tile_bounds(transform, col_off, row_off, width, height):
    """BBox do tile (em CRS do raster)."""
    x0, y0 = transform * (col_off, row_off)
    x1, y1 = transform * (col_off + width, row_off + height)
    # lembrar: y decresce para baixo; ajustar min/max:
    minx, maxx = min(x0, x1), max(x0, x1)
    miny, maxy = min(y0, y1), max(y0, y1)
    return minx, miny, maxx, maxy

In [ ]:
# ---------- Carregar metadados e validar ----------
years, paths = list_year_files(INPUT_DIR)
n_years = len(years)
print(f"Arquivos anuais encontrados: {n_years} ({years[0]}–{years[-1]})")

datasets = [rio.open(p) for p in paths]
ref = datasets[0]
print("CRS:", ref.crs)
print("Tamanho (px):", ref.width, "x", ref.height, "| bandas (esperado 12):", ref.count)
for ds in datasets[1:]:
    if not check_alignment(ref, ds):
        # Para aula, é melhor travar do que resamplar sem explicar
        for d in datasets: d.close()
        raise RuntimeError(f"Alinhamento diferente em {ds.name}. "
                           "Reexporte garantindo mesmo CRS, transform, resolução e extent.")
if ref.count != 12:
    for d in datasets: d.close()
    raise RuntimeError("Cada TIFF anual deve ter 12 bandas (meses). Verifique suas exportações.")

width, height = ref.width, ref.height
transform = ref.transform
# Nota: manter arquivos abertos (40) é ok; rasterio gerencia.

In [ ]:
# ---------- Iterador de janelas (tiles em px) ----------
def iter_windows(w, h, tile_px=TILE_PX):
    tid = 0
    for row_off in range(0, h, tile_px):
        for col_off in range(0, w, tile_px):
            tid += 1
            win_w = min(tile_px, w - col_off)
            win_h = min(tile_px, h - row_off)
            yield tid, Window(col_off, row_off, win_w, win_h), (col_off, row_off, win_w, win_h)

In [ ]:
# ---------- Processar UMA janela ----------
def process_window(tid, win):
    # Lê todos os anos (12 bandas) para a janela → cube shape (T, H, W)
    chunks = []
    for ds in datasets:
        arr = ds.read(indexes=list(range(1, 13)), window=win)  # (12, h, w)
        arr = arr.astype("float32")
        mask = (arr == NODATA)
        arr[~mask] = arr[~mask] / DATA_SCALE
        arr[mask] = np.nan
        chunks.append(arr)
    cube = np.concatenate(chunks, axis=0)  # (T=12*n_years, h, w)

    # Pular tiles vazios/escassos
    valid_frac = np.isfinite(cube).sum() / cube.size
    if valid_frac < VALID_FRAC_MIN:
        return {"tile_id": tid, "skipped": True}, None, None

    # Ruptures por pixel
    T, h, w = cube.shape
    flat = cube.reshape(T, -1).T  # (Npix, T)
    flat_ds = deseasonalize_series_matrix(flat, n_years=n_years)

    bp = np.full(flat.shape[0], -1, dtype=np.int32)
    for i in range(flat.shape[0]):
        bp[i] = first_break_1d(flat_ds[i, :], model=MODEL_RUPTURES)
    bp = bp.reshape(h, w)

    # Agregações por tile
    mask_bp = bp >= 0
    n_pixels = h * w
    n_break  = int(mask_bp.sum())
    frac     = (n_break / n_pixels) if n_pixels else 0.0

    # converter índices → ano/mês
    v = bp[mask_bp].ravel()
    years_v  = np.array([years[k // 12] for k in v], dtype=np.int32)
    months_v = np.array([1 + (k % 12) for k in v], dtype=np.int32)
    yyyymm_v = years_v * 100 + months_v

    # bbox do tile
    col_off, row_off, win_w, win_h = win.col_off, win.row_off, win.width, win.height
    minx, miny, maxx, maxy = tile_bounds(transform, col_off, row_off, win_w, win_h)

    # estatísticas do tile
    earliest = int(yyyymm_v.min()) if yyyymm_v.size else 0
    latest   = int(yyyymm_v.max()) if yyyymm_v.size else 0
    modal_year   = int(pd.Series(years_v).mode().iloc[0])  if years_v.size  else 0
    modal_yyyymm = int(pd.Series(yyyymm_v).mode().iloc[0]) if yyyymm_v.size else 0

    summary = {
        "tile_id": tid,
        "minx": minx, "miny": miny, "maxx": maxx, "maxy": maxy,
        "n_pixels": n_pixels, "n_break_pixels": n_break,
        "frac_breaks": frac,
        "earliest_yyyymm": earliest, "latest_yyyymm": latest,
        "modal_year": modal_year, "modal_yyyymm": modal_yyyymm
    }

    # contagens
    by_year = {}
    by_ym   = {}
    if years_v.size:
        y_u, c_y = np.unique(years_v,  return_counts=True)
        for yi, ci in zip(y_u, c_y): by_year[int(yi)] = int(ci)
    if yyyymm_v.size:
        ym_u, c_ym = np.unique(yyyymm_v, return_counts=True)
        for ymi, ci2 in zip(ym_u, c_ym): by_ym[int(ymi)] = int(ci2)

    return summary, by_year, by_ym

In [ ]:
# ---------- Rodar AMOSTRA (20 tiles) ----------
summaries = []
rows_year = []
rows_ym   = []

print(f"Rodando AMOSTRA de {SAMPLE_N_TILES} tiles para conferência...")
count_done = 0
for tid, win, _ in iter_windows(width, height, TILE_PX):
    summary, by_year, by_ym = process_window(tid, win)
    if summary is None:
        continue
    if not summary.get("skipped", False):
        summaries.append(summary)
        for y, c in (by_year or {}).items():
            rows_year.append({"tile_id": summary["tile_id"], "year": y, "count": c})
        for ym, c in (by_ym or {}).items():
            rows_ym.append({"tile_id": summary["tile_id"], "yyyymm": ym, "count": c})
    count_done += 1
    if count_done >= SAMPLE_N_TILES:
        break

df_sum_s  = pd.DataFrame(summaries)
df_year_s = pd.DataFrame(rows_year)
df_ym_s   = pd.DataFrame(rows_ym)

print("\n=== AMOSTRA: resumo por tile (top 10 por frac_breaks) ===")
if not df_sum_s.empty:
    print(df_sum_s.sort_values("frac_breaks", ascending=False).head(10).to_string(index=False))
else:
    print("(sem tiles válidos na amostra; ajuste TILE_PX, VALID_FRAC_MIN ou confirme os dados)")

print("\n=== AMOSTRA: contagem por ano (primeiras linhas) ===")
if not df_year_s.empty:
    print(df_year_s.sort_values(["tile_id","year"]).head(10).to_string(index=False))
else:
    print("(sem dados)")

print("\n=== AMOSTRA: contagem por ano-mês (primeiras linhas) ===")
if not df_ym_s.empty:
    print(df_ym_s.sort_values(["tile_id","yyyymm"]).head(10).to_string(index=False))
else:
    print("(sem dados)")


In [ ]:
# ---------- Rodar TODOS ----------
if not RUN_SAMPLE_ONLY:
    summaries = []
    rows_year = []
    rows_ym   = []
    for tid, win, _ in iter_windows(width, height, TILE_PX):
        summary, by_year, by_ym = process_window(tid, win)
        if summary is None:
            continue
        if not summary.get("skipped", False):
            summaries.append(summary)
            for y, c in (by_year or {}).items():
                rows_year.append({"tile_id": summary["tile_id"], "year": y, "count": c})
            for ym, c in (by_ym or {}).items():
                rows_ym.append({"tile_id": summary["tile_id"], "yyyymm": ym, "count": c})
        # (opcional) print progress a cada N tiles:
        if tid % 100 == 0:
            print(f"... {tid} tiles processados")

    df_sum = pd.DataFrame(summaries)
    df_year = pd.DataFrame(rows_year)
    df_ym   = pd.DataFrame(rows_ym)

    # Salvar CSVs
    csv1 = OUT_DIR / "tiles_with_breaks.csv"
    csv2 = OUT_DIR / "breaks_by_tile_year.csv"
    csv3 = OUT_DIR / "breaks_by_tile_yyyymm.csv"
    df_sum.to_csv(csv1, index=False)
    df_year.to_csv(csv2, index=False)
    df_ym.to_csv(csv3, index=False)
    print("\nCSVs salvos:")
    print(" -", csv1.name)
    print(" -", csv2.name)
    print(" -", csv3.name)



In [ ]:
from rasterio.windows import from_bounds

from rasterio.windows import from_bounds

def plot_tile_series(tile_row, show_inline=True, save_png=True, debug=False):
    import numpy as np, pandas as pd, matplotlib.pyplot as plt
    from IPython.display import display

    # --- IDs/valores robustos ---
    try:
        tid = int(tile_row["tile_id"])
    except Exception:
        tid = int(float(tile_row["tile_id"]))
    val = tile_row.get("modal_yyyymm", 0)
    modal_ym = 0
    if val is not None:
        try:
            if not (isinstance(val, float) and np.isnan(val)):
                modal_ym = int(val)
        except Exception:
            modal_ym = 0

    # --- Reconstruir janela pelo BBOX salvo no df_sum ---
    minx = float(tile_row["minx"]); miny = float(tile_row["miny"])
    maxx = float(tile_row["maxx"]); maxy = float(tile_row["maxy"])
    win = from_bounds(minx, miny, maxx, maxy, transform=transform).round_offsets().round_lengths()
    if win.width <= 0 or win.height <= 0:
        if debug: print(f"[plot] tile {tid}: janela inválida")
        return None

    # --- Série mediana mensal do tile ---
    series = []
    for ds in datasets:
        arr = ds.read(indexes=list(range(1,13)), window=win).astype("float32")  # (12,h,w)
        mask = (arr == NODATA)
        arr[~mask] = arr[~mask] / DATA_SCALE
        arr[mask]  = np.nan
        med = np.nanmedian(arr.reshape(12, -1), axis=1)  # (12,)
        series.append(med)
    ser = np.concatenate(series)  # (T,)

    # dessazonaliza (anomalia mensal)
    ser_ds = ser.copy()
    for m in range(12):
        clim = np.nanmean(ser[m::12])
        ser_ds[m::12] = ser_ds[m::12] - clim

    # --- Eixo temporal ---
    y0 = years[0]
    T  = ser_ds.size
    years_axis  = np.array([years[i//12] for i in range(T)], dtype=np.int64)
    months_axis = np.array([1 + (i%12) for i in range(T)], dtype=np.int64)
    yyyymm_axis = (years_axis*100 + months_axis).astype(np.int64)

    # --- Plot (inline + opcional salvar) ---
    fig, ax = plt.subplots(figsize=(9, 3.5))
    ax.plot(range(T), ser_ds)
    title = f"Tile {tid} — "
    if modal_ym > 0:
        pos = np.where(yyyymm_axis == modal_ym)[0]
        if pos.size:
            ax.axvline(int(pos[0]), linestyle="--")
            title += f"1º break (modal) em {modal_ym}"
        else:
            title += f"modal {modal_ym} fora do eixo"
    else:
        title += "sem modal identificável"
    ax.set_title(title)
    ax.set_xlabel(f"Meses desde jan/{y0}")
    ax.set_ylabel("NDVI (dessazonalizado)")
    fig.tight_layout()

    outp = PLOTS_DIR / f"tile{tid:05d}_series.png"
    if save_png:
        fig.savefig(outp, dpi=150)
        if debug: print(f"[plot] salvo: {outp}")

    if show_inline:
        display(fig)  # <-- mostra no notebook

    plt.close(fig)
    return outp if save_png else None

if not df_sum.empty:
    df_sum["tile_id"] = df_sum["tile_id"].astype("int64", errors="ignore")
    df_sum["modal_yyyymm"] = df_sum["modal_yyyymm"].fillna(0).astype("int64", errors="ignore")

    top = df_sum.sort_values("frac_breaks", ascending=False).head(TOP_N_PLOTS)
    print("Top para plotar:\n", top[["tile_id","modal_yyyymm","frac_breaks"]])

    for _, row in top.iterrows():
        plot_tile_series(row, show_inline=True, save_png=True, debug=False)
else:
    print("df_sum está vazio — nada para plotar.")